In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import Input, Activation, LeakyReLU, Dropout
from keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

import os

dataset_path = '/content/drive/MyDrive/archive (2)/brain_tumor_dataset'

folders = os.listdir(dataset_path)
print(folders)


['yes', 'no', 'test']


In [ ]:
subdirs = os.listdir(dataset_path)[:2]
for subdir in subdirs:
    print(f"{subdir} contains {len(os.listdir(dataset_path+'/'+subdir))} images")

yes contains 155 images
no contains 98 images


In [ ]:
def load_images(folder):

    imgs = []
    target = 0
    labels = []
    for i in os.listdir(folder):
        subdir = os.path.join(folder, i)
        for j in os.listdir(subdir):
            img_dir = os.path.join(subdir,j)
            try:
                img = cv2.imread(img_dir)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (128,128))
                imgs.append(img)
                labels.append(target)
            except:
                continue
        target += 1

    imgs = np.array(imgs)
    labels = np.array(labels)

    return imgs, labels




In [ ]:
data, labels = load_images(dataset_path)
data.shape, labels.shape


((254, 128, 128), (254,))

In [ ]:
norm_data = data / 255.
norm_data = np.expand_dims(norm_data, axis=3)
norm_data.shape, norm_data[0]

((254, 128, 128, 1),
 array([[[0.01176471],
         [0.01176471],
         [0.01176471],
         ...,
         [0.00784314],
         [0.00784314],
         [0.00784314]],
 
        [[0.01176471],
         [0.01176471],
         [0.01176471],
         ...,
         [0.00784314],
         [0.00784314],
         [0.00784314]],
 
        [[0.01176471],
         [0.01176471],
         [0.01176471],
         ...,
         [0.00784314],
         [0.00784314],
         [0.00784314]],
 
        ...,
 
        [[0.00784314],
         [0.00784314],
         [0.00784314],
         ...,
         [0.00784314],
         [0.00784314],
         [0.01176471]],
 
        [[0.00784314],
         [0.00784314],
         [0.00784314],
         ...,
         [0.00784314],
         [0.00784314],
         [0.01176471]],
 
        [[0.00784314],
         [0.00784314],
         [0.00784314],
         ...,
         [0.00392157],
         [0.00392157],
         [0.00784314]]]))

In [ ]:
# this is test code
tf.random.set_seed(21)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64,
                           kernel_size=3,
                           activation='relu',
                           input_shape=(128,128,1)),
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2,
                              padding='valid'),
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Conv2D(16,3,activation='relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

This the combination of logistic regression and CNN

In [ ]:
logistic_regression = tf.keras.layers.Dense(1, activation='sigmoid')

In [ ]:
combined_model = Sequential()
combined_model.add(model)
combined_model.add(logistic_regression)

combined_model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                       optimizer = tf.keras.optimizers.Adam(),
                       metrics = ["accuracy"])

In [ ]:
history = combined_model.fit(norm_data, labels, epochs = 55, validation_split = 0.20)

Epoch 1/55
7/7 [==============================] - 13s 251ms/step - loss: 0.6105 - accuracy: 0.7635 - val_loss: 0.9749 - val_accuracy: 0.0000e+00
Epoch 2/55
7/7 [==============================] - 0s 36ms/step - loss: 0.5937 - accuracy: 0.7635 - val_loss: 0.9832 - val_accuracy: 0.0000e+00
Epoch 3/55
7/7 [==============================] - 0s 36ms/step - loss: 0.5919 - accuracy: 0.7635 - val_loss: 0.9908 - val_accuracy: 0.0000e+00
Epoch 4/55
7/7 [==============================] - 0s 42ms/step - loss: 0.5903 - accuracy: 0.7635 - val_loss: 0.9985 - val_accuracy: 0.0000e+00
Epoch 5/55
7/7 [==============================] - 0s 40ms/step - loss: 0.5887 - accuracy: 0.7635 - val_loss: 1.0059 - val_accuracy: 0.0000e+00
Epoch 6/55
7/7 [==============================] - 0s 42ms/step - loss: 0.5871 - accuracy: 0.7635 - val_loss: 1.0124 - val_accuracy: 0.0000e+00
Epoch 7/55
7/7 [==============================] - 0s 43ms/step - loss: 0.5858 - accuracy: 0.7635 - val_loss: 1.0198 - val_accuracy: 0.0000e+

In [ ]:
result = combined_model.evaluate(norm_data, labels, verbose=0)

In [ ]:
print(f"Accuracy on Evaluation of the CNN and Logistic Regression Model: {result[1]*100:.2f}%\nLoss: {result[0]:.4f}")

Accuracy on Evaluation of the CNN and Logistic Regression Model: 61.02%
Loss: 0.7014


In [ ]:
# Get predictions
predictions = model.predict(norm_data)
y_pred = np.round(predictions).flatten().astype(int)


# Compute confusion matrix
cm = confusion_matrix(labels, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)



8/8 [==============================] - 0s 5ms/step
Confusion Matrix:
[[  0 155   0]
 [  0  98   0]
 [  0   1   0]]


This the CNN only layer of the model

In [ ]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
              # optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [ ]:
# history = model.fit(norm_data, labels, epochs = 50, validation_split = 0.20)

In [ ]:
# result = model.evaluate(norm_data, labels, verbose=0)
print(f"Accuracy on Evaluation of the CNN only model: {result[1]*100:.2f}%\nLoss: {result[0]:.4f}")

Accuracy on Evaluation of the CNN only model: 61.02%
Loss: 0.7014


In [ ]:


# # Get predictions
# predictions = model.predict(norm_data)
# y_pred = np.round(predictions).flatten().astype(int)

# # Compute confusion matrix
# cm = confusion_matrix(labels, y_pred)

# # Print confusion matrix
# print("Confusion Matrix:")
# print(cm)
